<a href="https://colab.research.google.com/github/Gapilesh/BlinkKey_FinalProject/blob/main/Eyeblink8_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


**2. Extract Dataset**

In [ ]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/Dataset/Eyeblink/eyeblink8.zip'
extract_path = '/content/drive/MyDrive/Dataset/Eyeblink'

if not os.path.exists(os.path.join(extract_path, 'eyeblink8')):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("Dataset unzipped successfully!")
else:
    print("Dataset already exists, skipping extraction.")


Dataset unzipped successfully!


**3. Check Dataset Structure**

In [ ]:
dataset_path = '/content/drive/MyDrive/Dataset/Eyeblink/eyeblink8'
folders = sorted(os.listdir(dataset_path))
print("Folders in the dataset:", folders)

sample_folder = os.path.join(dataset_path, '1')
sample_files = os.listdir(sample_folder)
print("Files in folder '1':", sample_files)


Folders in the dataset: ['1', '10', '11', '2', '3', '4', '8', '9']
Files in folder '1': ['26122013_223310_cam.avi', '26122013_223310_cam.tag', '26122013_223310_cam.txt']


**4. Install & Import OpenCV**

In [ ]:
!pip install opencv-python

import cv2
import os
import numpy as np


**5. Extract Frames Efficiently**

In [ ]:
def extract_frames(video_path, output_folder, frame_skip=10, frame_limit=500):
    os.makedirs(output_folder, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    extracted_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret or extracted_count >= frame_limit:
            break

        if frame_count % frame_skip == 0:  # Skip frames to reduce memory load
            frame_path = os.path.join(output_folder, f"frame_{extracted_count:04d}.jpg")
            cv2.imwrite(frame_path, frame)
            extracted_count += 1

        frame_count += 1

    cap.release()
    print(f"Extracted {extracted_count} frames from {video_path}")

video_path = os.path.join(dataset_path, '1', '1.mp4')
output_folder = '/content/drive/MyDrive/Dataset/Eyeblink/frames/1'
extract_frames(video_path, output_folder)


Extracted 0 frames from /content/drive/MyDrive/Dataset/Eyeblink/eyeblink8/1/1.mp4


**6. Parse Annotations**

In [ ]:
def parse_annotations(annotation_path):
    with open(annotation_path, 'r') as file:
        lines = file.readlines()

    timestamps = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) == 2:
            timestamps.append(float(parts[1]))  # Assuming second value is timestamp

    return timestamps

annotation_path = os.path.join(dataset_path, '1', '1.txt')
timestamps = parse_annotations(annotation_path)
print("Sample timestamps:", timestamps[:10])


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Dataset/Eyeblink/eyeblink8/1/1.txt'

**7. Prepare Frames & Labels**

In [ ]:
def load_frames_and_labels(folder, frame_limit=500):
    frames, labels = [], []
    annotation_path = os.path.join(folder, '1.txt')

    if not os.path.exists(annotation_path):
        print("Annotation file missing for:", folder)
        return np.array([]), np.array([])

    timestamps = parse_annotations(annotation_path)

    for i in range(min(len(timestamps), frame_limit)):
        frame_path = os.path.join(folder, f"frame_{i:04d}.jpg")
        if os.path.exists(frame_path):
            frame = cv2.imread(frame_path)
            frame = cv2.resize(frame, (224, 224)) / 255.0  # Normalize
            frames.append(frame)
            labels.append(1 if timestamps[i] > 0 else 0)  # Example thresholding

    return np.array(frames), np.array(labels)

frames, labels = load_frames_and_labels('/content/drive/MyDrive/Dataset/Eyeblink/frames/1')
print("Frames shape:", frames.shape)
print("Labels shape:", labels.shape)


**8. Define a Lightweight Model**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed

model = Sequential([
    TimeDistributed(Conv2D(16, (3,3), activation='relu'), input_shape=(None, 224, 224, 3)),
    TimeDistributed(MaxPooling2D((2,2))),
    TimeDistributed(Flatten()),
    LSTM(32, return_sequences=False),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


**9. Train Model in Small Batches**

In [ ]:
batch_size = 100  # Smaller batch size to fit in RAM
frames = np.expand_dims(frames, axis=0)  # Add batch dimension
labels = np.expand_dims(labels, axis=0)

for epoch in range(3):  # Reduce epochs to avoid crashes
    print(f"Training Epoch {epoch + 1}...")
    model.fit(frames, labels, epochs=1, batch_size=batch_size, verbose=1)

# Save model
model.save('/content/drive/MyDrive/eyeblink_model.h5')


**10. Evaluate Model**

In [ ]:
loss, accuracy = model.evaluate(frames, labels)
print(f"Loss: {loss}, Accuracy: {accuracy}")
